##Ingest Drivers json file

###1.read json


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                  StructField("surname", StringType(), True)])

In [0]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                      StructField("driverRef", StringType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("code", StringType(), True),
                                      StructField("name", name_schema),
                                      StructField("dob", DateType(), True),
                                      StructField("nationality", StringType(), True),
                                      StructField("url", StringType(), True)])

In [0]:
drivers_df=spark.read.schema(drivers_schema).json("abfss://raw@formula1dlbtg.dfs.core.windows.net/drivers.json")
display(drivers_df)

In [0]:
drivers_df.printSchema()

##2. rename columns and add new columns

In [0]:
from pyspark.sql.functions import current_timestamp,lit,col,concat

In [0]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("ingestion_date", current_timestamp()).withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))
display(drivers_with_columns_df)

drop the unwanted column

In [0]:
drivers_final_df=drivers_with_columns_df.drop(col("url"))

save


In [0]:
drivers_final_df.write.mode("overwrite").parquet("abfss://processed@formula1dlbtg.dfs.core.windows.net/drivers")

In [0]:
display(spark.read.parquet("abfss://processed@formula1dlbtg.dfs.core.windows.net/drivers"))